In [1]:
'''
This is the second step after receiving the data from the youtube api.
This files extract the data and safes them once as csv and once as nested dictionary
'''
import json
import csv
import pandas as pd


In [2]:
with open('data/raw/comments_dict_reichelt.json', 'r') as json_file:
    # Load the JSON data
    data_reichelt = json.load(json_file)
    
with open('data/raw/video_stat_reichelt.json', 'r') as json_file:
    # Load the JSON data
    stat_data_reichelt = json.load(json_file)

In [3]:
with open('data/raw/video_stat_dw.json', 'r') as json_file:
    # Load the JSON data
    stat_data_dw = json.load(json_file)
    
with open('data/raw/comments_dict_dw.json', 'r') as json_file:
    # Load the JSON data
    data_dw = json.load(json_file)

In [4]:
with open('data/raw/video_stat_jung.json', 'r') as json_file:
    # Load the JSON data
    stat_data_jung = json.load(json_file)
    
with open('data/raw/comments_dict_jung.json', 'r') as json_file:
    # Load the JSON data
    data_jung = json.load(json_file)

In [5]:
df = pd.read_csv("data/raw/vids.csv", index_col=False)

In [6]:
def cleanData(ChannelId,comments_data,vid):
    '''
    This function takes all json junks from the youtube api request and extracts important data to merge into a csv later on 
    '''
    #list for comment csv
    text = []
    publishedAt = []
    authorID = []
    likeCount = [] 
    totalReplyCount = []
    parentID = []
    commentID = []
    videoID = []
    channelId = []
    username = []

    #list for video csv
    videoPublishedAt = []
    videolikeCount = [] 
    title = []
    description = []
    viewCount = []
    likeCount = []
    commentCount = []
    videochannelId = []
    videovideoID = []
    channelName = []
    transcript = []

    yt_data = {
       ChannelId: {}
    }

    for key, value in comments_data.items():
        comments = {}

        for i in value:
            comment_id = i["id"]
            try:
                comment_data = {
                    "publishedAt": i["snippet"]['topLevelComment']["snippet"]['publishedAt'],
                    "text": i["snippet"]['topLevelComment']["snippet"]['textOriginal'],
                    "authorID": i["snippet"]['topLevelComment']["snippet"]['authorChannelId']['value'],
                    "likeCount": i["snippet"]['topLevelComment']["snippet"]['likeCount'],
                    "totalReplyCount": i["snippet"]['totalReplyCount'],
                    "parentID": 0
                }

                comments[comment_id] = comment_data

                text.append(i["snippet"]['topLevelComment']["snippet"]['textOriginal'])
                publishedAt.append(i["snippet"]['topLevelComment']["snippet"]['publishedAt'])
                authorID.append(i["snippet"]['topLevelComment']["snippet"]['authorChannelId']['value'])
                likeCount.append(i["snippet"]['topLevelComment']["snippet"]['likeCount'])
                totalReplyCount.append( i["snippet"]['totalReplyCount'])
                parentID.append(0)
                commentID.append(i["id"])
                videoID.append(key)
                channelId.append(ChannelId)
                username.append(i["snippet"]['topLevelComment']["snippet"]['authorDisplayName'])
            except:
                
                comment_data = {
                    "publishedAt": i["snippet"]['topLevelComment']["snippet"]['publishedAt'],
                    "text": i["snippet"]['topLevelComment']["snippet"]['textOriginal'],
                    "authorID": 0,
                    "likeCount": i["snippet"]['topLevelComment']["snippet"]['likeCount'],
                    "totalReplyCount": i["snippet"]['totalReplyCount'],
                    "parentID": 0
                }

                comments[comment_id] = comment_data

                text.append(i["snippet"]['topLevelComment']["snippet"]['textOriginal'])
                publishedAt.append(i["snippet"]['topLevelComment']["snippet"]['publishedAt'])
                authorID.append(0)
                likeCount.append(i["snippet"]['topLevelComment']["snippet"]['likeCount'])
                totalReplyCount.append( i["snippet"]['totalReplyCount'])
                parentID.append(0)
                commentID.append(i["id"])
                videoID.append(key)
                channelId.append(ChannelId)
                username.append(i["snippet"]['topLevelComment']["snippet"]['authorDisplayName'])
                


            #check for replies and extract them
            if i["snippet"]['totalReplyCount'] > 0:
                for x in i["replies"]["comments"]:
                    try:
                        reply_id = x["id"]
                        reply_data = {
                            "publishedAt": x["snippet"]['publishedAt'],
                            "text": x["snippet"]['textOriginal'],
                            "authorID": x["snippet"]['authorChannelId']['value'],
                            "likeCount": x["snippet"]['likeCount'],
                            "totalReplyCount": 0,
                            "parentID": x["snippet"]['parentId']
                        }

                        comments[reply_id] = reply_data

                        text.append(x["snippet"]['textOriginal'])
                        publishedAt.append(x["snippet"]['publishedAt'])
                        authorID.append(x["snippet"]['authorChannelId']['value'])
                        likeCount.append(x["snippet"]['likeCount'])
                        totalReplyCount.append(0)
                        parentID.append(x["snippet"]['parentId'])
                        commentID.append(reply_id)
                        videoID.append(key)
                        channelId.append(ChannelId)
                        username.append(x["snippet"]['authorDisplayName'])

                    except:
                        pass
    
    for key, value in vid.items():
        if value["items"][0]["id"] == key:
            j = value["items"][0]
            try:
                vid_stat = {
                    "publishedAt": j["snippet"]['publishedAt'],
                    "title": j["snippet"]['title'],
                    "description": j["snippet"]['description'],
                    "viewCount": j["statistics"]['viewCount'],
                    "likeCount": j["statistics"]['likeCount'],
                    "commentCount": j["statistics"]['commentCount']
                }


                videoPublishedAt.append(j["snippet"]['publishedAt'])
                title.append(j["snippet"]['title'])
                description.append(j["snippet"]['description'])
                viewCount.append(j["statistics"]['viewCount'])
                videolikeCount.append(j["statistics"]['likeCount'])
                commentCount.append(j["statistics"]['commentCount'])
                videochannelId.append(ChannelId)
                videovideoID.append(key)
                channelName.append(j["snippet"]['channelTitle'])
                transcript.append(df.loc[df['video_id'] == key, 'transcript'].values[0])

                    
                
            except:
                vid_stat = {
                    "publishedAt": j["snippet"]['publishedAt'],
                    "title": j["snippet"]['title'],
                    "description": j["snippet"]['description'],
                    "viewCount": j["statistics"]['viewCount'],
                    "likeCount": j["statistics"]['likeCount'],
                    "commentCount": len(comments_data[key])
                }
                    
                videoPublishedAt.append(j["snippet"]['publishedAt'])
                title.append(j["snippet"]['title'])
                description.append(j["snippet"]['description'])
                viewCount.append(j["statistics"]['viewCount'])
                videolikeCount.append(j["statistics"]['likeCount'])
                commentCount.append(len(comments_data[key]))
                videochannelId.append(ChannelId)
                videovideoID.append(key)
                channelName.append(j["snippet"]['channelTitle'])
                transcript.append(df.loc[df['video_id'] == key, 'transcript'].values[0])
                    
                    

        yt_data[ChannelId][key] = [vid_stat, comments]

    return yt_data, commentID, text, publishedAt, authorID, likeCount, totalReplyCount, parentID, videoID, channelId, username,  videoPublishedAt, videolikeCount, title, description, viewCount, videolikeCount, commentCount, videochannelId, videovideoID,channelName, transcript 



In [7]:
'''
Loop through every data and extract values, then merge listws to get combined data
'''
    
channel_ids = ["UCcoQ3WG2J_Xjwwyt-sJqh-w", 'UCMIgOXM2JEQ2Pv2d0_PVfcg', 'UCv1WDP5EiipMQ__C4Cg6aow']
comments_data =[data_reichelt,  data_dw, data_jung]
vid_data = [stat_data_reichelt,stat_data_dw, stat_data_jung]

merged_commentID = []
merged_text = []
merged_publishedAt = []
merged_authorID = []
merged_likeCount = []
merged_totalReplyCount = []
merged_parentID = []
merged_videoID = []
merged_channelId = []
merged_username = []
merged_videoPublishedAt = []
merged_videolikeCount = []
merged_title = []
merged_description = []
merged_viewCount = []
merged_commentCount = []
merged_videochannelId = []
merged_videovideoID = []
merged_channelName = []
merged_transcript = []


for i in range(len(channel_ids)):
    print(i)
    yt_data, commentID, text, publishedAt, authorID, likeCount, totalReplyCount, parentID, videoID, channelId, username, videoPublishedAt, videolikeCount, title, description, viewCount, videolikeCount, commentCount, videochannelId, videovideoID, channelName, transcript = cleanData(channel_ids[i], comments_data[i], vid_data[i])

    # Merge the lists
    merged_commentID.extend(commentID)
    merged_text.extend(text)
    merged_publishedAt.extend(publishedAt)
    merged_authorID.extend(authorID)
    merged_likeCount.extend(likeCount)
    merged_totalReplyCount.extend(totalReplyCount)
    merged_parentID.extend(parentID)
    merged_videoID.extend(videoID)
    merged_channelId.extend(channelId)
    merged_username.extend(username)
    merged_videoPublishedAt.extend(videoPublishedAt)
    merged_videolikeCount.extend(videolikeCount)
    merged_title.extend(title)
    merged_description.extend(description)
    merged_viewCount.extend(viewCount)
    merged_commentCount.extend(commentCount)
    merged_videochannelId.extend(videochannelId)
    merged_videovideoID.extend(videovideoID)
    merged_channelName.extend(channelName)
    merged_transcript.extend(transcript)



0
1
2


In [8]:
# Create a list of rows, where each row is a list of attributes
rows = list(zip(merged_commentID, 
                merged_text, 
                merged_publishedAt, 
                merged_authorID, 
                merged_likeCount, 
                merged_totalReplyCount,
                merged_parentID, 
                merged_videoID, 
                merged_channelId,
                merged_username))

# Define the CSV file path
csv_file = "data/raw/comments.csv"

# Write the rows to the CSV file with UTF-8 encoding
with open(csv_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["commentID", "text", "publishedAt", "authorID", "likeCount", "totalReplyCount", "parentID", "videoID", "channelId", "username"])  # Write header
    writer.writerows(rows)  # Write data rows

print("CSV file created successfully.")

CSV file created successfully.


In [9]:
# Create a list of rows, where each row is a list of attributes
rows = list(
    zip(
        merged_videoPublishedAt,
        merged_videolikeCount,
        merged_title,
        merged_description,
        merged_viewCount,
        merged_videolikeCount,
        merged_commentCount,
        merged_videochannelId,
        merged_videovideoID,
        merged_channelName,
        merged_transcript
    )
)

# Define the CSV file path
csv_file = "data/raw/videos.csv"

# Write the rows to the CSV file with UTF-8 encoding
with open(csv_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(
        [
            "videoPublishedAt",
            "videolikeCount",
            "title",
            "description",
            "viewCount",
            "likeCount",
            "commentCount",
            "videochannelId",
            "videovideoID",
            "channelName",
            "transcript"
        ]
    )  # Write header
    writer.writerows(rows)  # Write data rows

print("CSV file created successfully.")

CSV file created successfully.
